## Import

In [1]:
from ilthermo.models import *
ions = session.query(Ion)
mols = session.query(Molecule)

## Canonicalize SMILES

In [2]:
for ion in ions.filter(Ion.ignored==False):
    m = pybel.readstring('smi', ion.smiles)
    ion.smiles = m.write('can').strip()
session.commit()

## Check molcular formula

In [10]:
for mol in mols:
    if not mol.cation.ignored and not mol.anion.ignored:
        smiles = mol.cation.smiles + '.' + mol.anion.smiles
        m = pybel.readstring('smi', smiles)
        if mol.formula != m.formula:
            print(mol, mol.formula, m.formula, mol.cation_id, mol.anion_id)

## Check number of papers for each ion

In [ ]:
for ion in ions.filter(Ion.ignored==False):
    ps = set()
    for mol in ion.molecules:
        for data in mol.datas:
            ps.add(data.paper)
    print(ion, len(ps))
    ion.n_paper = len(ps)
session.commit()

## Check duplicated ions

In [58]:
smiles_ion = dict()
for ion in ions.filter(Ion.ignored==False).order_by(Ion.n_paper.desc()):
    if ion.smiles not in smiles_ion.keys():
        smiles_ion[ion.smiles] = ion
    else:
        print(ion, smiles_ion[ion.smiles])
        ion.duplicate = smiles_ion[ion.smiles].id
session.commit()

<Ion -1: 22 bis(trifluoromethylsulfonyl)imide> <Ion -1: 81 bis[(trifluoromethyl)sulfonyl]imide>
<Ion -1: 87 bis((trifluoromethyl)sulfonyl)amide> <Ion -1: 81 bis[(trifluoromethyl)sulfonyl]imide>
<Ion +1: 374 1-methyl-3-octylimidazolium> <Ion +1: 265 1-octyl-3-methylimidazolium>
<Ion -1: 310 bis(trifluoromethylsulfonyl)amide> <Ion -1: 81 bis[(trifluoromethyl)sulfonyl]imide>
<Ion -1: 93 1,1,1-trifluoro-N-[(trifluoromethyl)sulfonyl]methanesulfonamide> <Ion -1: 81 bis[(trifluoromethyl)sulfonyl]imide>
<Ion -1: 740 methylsulfate> <Ion -1: 840 methyl sulfate>
<Ion +1: 10 1-ethyl-3-methyl-1H-imidazolium> <Ion +1: 105 1-ethyl-3-methylimidazolium>
<Ion -1: 97 bis(trifluoromethanesulfonyl)imide> <Ion -1: 81 bis[(trifluoromethyl)sulfonyl]imide>
<Ion -1: 98 ethylsulfate> <Ion -1: 841 ethyl sulfate>
<Ion -1: 129 bis((trifluoromethyl)sulfonyl)imide> <Ion -1: 81 bis[(trifluoromethyl)sulfonyl]imide>
<Ion -1: 34 tricyanomethane> <Ion -1: 101 tricyanomethanide>
<Ion +1: 445 tetradecyl(trihexyl)phosphonium

## Check number of papers for unique ion

In [3]:
for ion in ions.filter(Ion.ignored==False).order_by(Ion.n_paper.desc()):
    if ion.duplicate is not None:
        ion.times = 0
    else:
        ion.times = ion.n_paper
        for eqi in ions.filter(Ion.duplicate==ion.id):
            ion.times += eqi.n_paper
session.commit()

## Show number of heavy atoms

In [11]:
for ion in ions.filter(Ion.selected==True):
    print(ion.n_heavy, ion)

10 <Ion +1: 2 1-butyl-3-methylimidazolium>
9 <Ion +1: 5 1-methyl-1-propylpyrrolidinium>
9 <Ion -1: 6 bis(fluorosulfonyl)amide>
12 <Ion +1: 8 1-hexylpyridinium>
8 <Ion +1: 11 1-ethylpyridinium>
10 <Ion +1: 12 1-butylpyridinium>
10 <Ion +1: 14 1-butyl-1-methylpyrrolidinium>
7 <Ion -1: 20 hexafluorophosphate>
7 <Ion -1: 21 trifluoroacetate>
4 <Ion -1: 23 nitrate>
5 <Ion -1: 24 tetrafluoroborate>
4 <Ion -1: 28 acetate>
5 <Ion -1: 29 dicyanamide>
1 <Ion -1: 30 chloride>
8 <Ion -1: 33 trifluoromethanesulfonate>
7 <Ion -1: 36 dimethylphosphate>
3 <Ion -1: 38 thiocyanate>
9 <Ion -1: 41 tetracyanoborate>
5 <Ion -1: 42 perchlorate>
5 <Ion -1: 54 methanesulfonate>
3 <Ion -1: 57 formate>
15 <Ion -1: 81 bis[(trifluoromethyl)sulfonyl]imide>
7 <Ion -1: 101 tricyanomethanide>
8 <Ion +1: 105 1-ethyl-3-methylimidazolium>
5 <Ion -1: 106 propionate>
8 <Ion +1: 126 butyltrimethylammonium>
6 <Ion +1: 172 diethylmethylammonium>
12 <Ion +1: 264 1-hexyl-3-methylimidazolium>
11 <Ion +1: 327 1-butyl-1-methylpipe

## Select molecules

In [46]:
for mol in mols:
    mol.selected = False
    cation = mol.cation
    anion = mol.anion
    if cation.duplicate is not None:
        cation = ions.get(cation.duplicate)
    if anion.duplicate is not None:
        anion = ions.get(anion.duplicate)
    if cation.selected and anion.selected:
        mol.selected = True
        print(mol.cation.category, mol.anion.category, mol)
session.commit()

cIm aSI <Molecule: 1 1-butyl-3-methylimidazolium bis(fluorosulfonyl)imide>
cPyrr aSI <Molecule: 3 1-methyl-1-propylpyrrolidinium bis(fluorosulfonyl)amide>
cPy aSI <Molecule: 5 1-hexylpyridinium bis(fluorosulfonyl)amide>
cIm aSI <Molecule: 7 1-ethyl-3-methyl-1H-imidazolium bis(fluorosulfonyl)amide>
cPy aSI <Molecule: 8 1-ethylpyridinium bis(fluorosulfonyl)amide>
cPy aSI <Molecule: 9 1-butylpyridinium bis(fluorosulfonyl)amide>
cPyrr aSI <Molecule: 11 1-butyl-1-methylpyrrolidinium bis(fluorosulfonyl)amide>
cIm aXP6 <Molecule: 17 1-butyl-3-methylimidazolium hexafluorophosphate>
cIm aCO2 <Molecule: 18 1-butyl-3-methylimidazolium trifluoroacetate>
cIm aSI <Molecule: 19 1-butyl-3-methylimidazolium bis(trifluoromethylsulfonyl)imide>
cIm aNO3 <Molecule: 20 1-butyl-3-methylimidazolium nitrate>
cIm aXB4 <Molecule: 21 1-butyl-3-methylimidazolium tetrafluoroborate>
cIm aCO2 <Molecule: 25 1-butyl-3-methylimidazolium acetate>
cIm aNC[N] <Molecule: 26 1-butyl-3-methylimidazolium dicyanamide>
cIm a0Cl 

## Fit density and viscosity for selected molecuels

In [36]:
mols.update({'fit':None})
session.commit()
#for mol in mols.filter(Molecule.selected==True):
#    print(mol)
#    try:
#        score = mol.fit_density()
#        print(score)
#    except:
#        pass
#    print(mol.fit)

## Misc

In [6]:
ion = ions.get(185)
ion.molecules.all()

[<Molecule: 210 triethylammonium acetate>,
 <Molecule: 1289 triethylammonium hydrogen sulfate>,
 <Molecule: 1304 triethylammonium dihydrogen phosphate>]

In [5]:
mol = mols.get(1)
print(mol.cation)
print(mol.anion)

<Ion +1: 2 1-butyl-3-methylimidazolium>
<Ion -1: 3 bis(fluorosulfonyl)imide>


In [7]:
mol = mols.get(1)
vis = session.query(Property).filter(Property.name=='Viscosity').first()
datas = mol.datas.filter(Data.property==vis)
print(datas.all())

[<Data: Viscosity: 273.1 101.0 0.098800>, <Data: Viscosity: 278.1 101.0 0.076500>, <Data: Viscosity: 283.1 101.0 0.060400>, <Data: Viscosity: 288.1 101.0 0.048600>, <Data: Viscosity: 293.1 101.0 0.039700>, <Data: Viscosity: 298.1 101.0 0.032900>, <Data: Viscosity: 303.1 101.0 0.027600>, <Data: Viscosity: 313.1 101.0 0.020220>, <Data: Viscosity: 323.1 101.0 0.015350>, <Data: Viscosity: 333.1 101.0 0.012010>, <Data: Viscosity: 343.1 101.0 0.009650>, <Data: Viscosity: 353.1 101.0 0.007920>, <Data: Viscosity: 363.1 101.0 0.006630>]


In [14]:
import pybel
s = pybel.Smarts('n1cncc1[#6]')
for ion in ions.filter(Ion.smiles!=None):
    py_mol = pybel.readstring('smi', ion.smiles)
    if s.findall(py_mol) != []:
        print(ion)   

<Ion -1: 75 4,5-dicyano-2-(trifluoromethyl)imidazolide>
<Ion +1: 367 1-butyl-2,3,4,5-tetramethylimidazolium>
<Ion +1: 368 1,2,3,4,5-pentamethylimidazolium>
<Ion +1: 420 3-butyl-1,2,4,5-tetramethyl-1H-imidazol-3-ium>
<Ion +1: 692 1-butyl-1H-benzimidazolium>
<Ion -1: 761 L-histidinate>
<Ion -1: 793 4,5-dicyanoimidazolide>
